In [3]:
import sys

# INF in infinty
INF = sys.maxsize


# Helper function for sorting based on distance
def distance(x):
    return x[1]


def djikstra(graph, source, destination):
    size = len(graph)  # size -> Number of nodes in the graph
    parent = [0] * size
    dist = [INF] * size  # dist -> List of distances from the source
    dist[source] = 0  # Distance of source from source is 0
    parent[source] = 0
    visited = [False] * size
    # visited -> List of nodes that have been added to the tree
    visited[0] = True
    queue = []
    # queue -> List of [node, distance] that have not yet been added to the tree, but seen before
    queue.append([source, 0])

    # Loop till there are nodes that are not added in the tree
    while (len(queue) > 0):
        # i is the index of the node that will be added to the tree in this iteration
        i = queue[0][0]
        # prevDist is the minimum distance from source to i
        prevDist = queue[0][1]
        # Remove the added node from the queue
        del queue[0]
        # indices -> List of [node, distance from source] of the indices adjacent to i
        indices = []
        # i has been added to the tree => visited[i] = True
        visited[i] = True
        '''
        print("dist:", dist)
        print("visited:", visited)
        print("queue:", queue)
        print("\n")
        '''

        # Iterating through graph[i] to find adjacent indices
        for j in range(size):
            # graph[i][j] < INF     => There is an edge between i and j
            # visited[j] == False   => The node has not been already added to the tree
            if graph[i][j] < INF and visited[j] == False:
                # Choose the smaller distance
                if graph[i][j] + prevDist < dist[j]:
                    indices.append([j, graph[i][j] + prevDist])

        # Sort the adjacent indices by their distance from the source
        indices.sort(key=distance)
        # If there are adjacent vertices,
        if (len(indices) > 0):
            # Update the distance from the source
            for x in indices:
                dist[x[0]] = x[1]
                parent[x[0]] = i
            # Add the adjacent vertices to the queue
            queue.extend(indices)

    path = [destination]

    while (path[-1] != source):
        path.append(parent[path[-1]])

    path.reverse()

    return path


def fragmentation(d, mtu, header=20):
    sequence = 0
    offset = 0
    size = d.length
    size -= header
    mtu -= header
    fragments = []

    while (size > 0):
        if (size >= mtu):
            fragments.append(
                Datagram(sequence, d.identifier, mtu + header, d.df, 1,
                         offset // 8))
            size -= mtu
            offset += mtu
        else:
            fragments.append(
                Datagram(sequence, d.identifier, size + header, d.df, 0,
                         offset // 8))
            size = 0
        sequence += 1

    return fragments


class Datagram:

    def __init__(self, s, _id, length, df, mf, offset):
        self.sequence = s
        self.identifier = _id
        self.length = length
        self.df = df
        self.mf = mf
        self.offset = offset

    def update(self, s, _id, length, df, mf, offset):
        self.sequence = s
        self.identifier = _id
        self.length = length
        self.df = df
        self.mf = mf
        self.offset = offset

    def display(self):
        print("Sequence: ",
              self.sequence,
              "\nIdentifier: ",
              self.identifier,
              "\nLength: ",
              self.length,
              "\nDo Not Fragment: ",
              self.df,
              "\nMore Frames: ",
              self.mf,
              "\nOffset: ",
              self.offset,
              sep="")


def main():
    d = Datagram(0, 345, 5140, 0, 0, 0)
    print("DATAGRAMS AT A")
    d.display()
    print("\n")

    graph = [
        # A R1 R2 R3 B
        [0, 0, INF, INF, INF],
        [0, 0, 3, 6, INF],
        [INF, 3, 0, 1, INF],
        [INF, 6, 1, 0, 0],
        [INF, INF, INF, 0, 0]
    ]

    source = 0
    destination = 4
    path = djikstra(graph, source, destination)
#     print(path)

    mtu_graph = [[INF, 1500, INF, INF, INF], [1500, INF, 620, 620, INF],
                 [INF, 620, INF, 620, INF], [INF, 620, 620, INF, 620],
                 [INF, INF, INF, 620, INF]]

    for i in range(1, len(path)):
        fragments = fragmentation(d, mtu_graph[i - 1][i])
        print("\n\nDATAGRAMS AT", i)
        print("\n")
        for fragment in fragments:
            fragment.display()
            print("\n")


main()

DATAGRAMS AT A
Sequence: 0
Identifier: 345
Length: 5140
Do Not Fragment: 0
More Frames: 0
Offset: 0




DATAGRAMS AT 1


Sequence: 0
Identifier: 345
Length: 1500
Do Not Fragment: 0
More Frames: 1
Offset: 0


Sequence: 1
Identifier: 345
Length: 1500
Do Not Fragment: 0
More Frames: 1
Offset: 185


Sequence: 2
Identifier: 345
Length: 1500
Do Not Fragment: 0
More Frames: 1
Offset: 370


Sequence: 3
Identifier: 345
Length: 700
Do Not Fragment: 0
More Frames: 0
Offset: 555




DATAGRAMS AT 2


Sequence: 0
Identifier: 345
Length: 620
Do Not Fragment: 0
More Frames: 1
Offset: 0


Sequence: 1
Identifier: 345
Length: 620
Do Not Fragment: 0
More Frames: 1
Offset: 75


Sequence: 2
Identifier: 345
Length: 620
Do Not Fragment: 0
More Frames: 1
Offset: 150


Sequence: 3
Identifier: 345
Length: 620
Do Not Fragment: 0
More Frames: 1
Offset: 225


Sequence: 4
Identifier: 345
Length: 620
Do Not Fragment: 0
More Frames: 1
Offset: 300


Sequence: 5
Identifier: 345
Length: 620
Do Not Fragment: 0
More Frames